# NMMC Grievance Analytics — Senior DS Notebook

**Goal:** Build an end‑to‑end, production‑oriented analysis of NMMC grievances (quality checks → KPI layer → text mining → sub‑topic taxonomy → modeling ideas → deployment/monitoring plan).

> **Data source:** `channel-wise-grievance-data.xlsx`

## Headline snapshot (from the current file)
- **Rows:** 11,103
- **Unique grievance IDs:** 11,103
- **Created date range:** 2024-10-11 04:21 → 2025-12-31 10:56
- **Closed rate (by current status):** 96.39%

**Top departments (count):**
- Encroachment: 1,932
- City Engineer: 1,617
- Water Supply: 1,316
- Solid Waste Management: 1,211
- Electrical: 972

**Top wards (count):**
- Belapur: 1,746
- Koparkhairane: 1,662
- Airoli: 1,463
- Ghansoli: 1,370
- Vashi: 1,283

**Top missing fields (% rows):**
- Complaint Description: 5.63%
- Closing Remark: 4.52%
- Mobile No.: 0.26%
- Reported by User Name: 0.07%
- Complaint Location: 0.02%

---

### How to use this notebook
1. Run cells top‑to‑bottom.
2. Replace file path if needed.
3. This notebook **masks PII** (mobile numbers + user names) in outputs by default.


In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 120)


## 1) Load data + repair header row

This Excel has a **two‑row header structure**: a banner row + a row that contains the real column names.
We read with `header=1`, then promote the first data row into headers.

We also parse `Created Date` into a timestamp column `_created_at`.


In [ ]:
# File path
SRC_PATH = r"/mnt/data/channel-wise-grievance-data.xlsx"

raw = pd.read_excel(SRC_PATH, header=1)
headers = raw.iloc[0].tolist()

df = raw.iloc[1:].copy()
df.columns = [str(h).strip() for h in headers]
df.reset_index(drop=True, inplace=True)

df["_created_at"] = pd.to_datetime(df["Created Date"].astype(str), dayfirst=True, errors="coerce")

df.shape, df.columns.tolist()


## 2) Data contract & schema checks (senior DS hygiene)

We treat the dataset like an **API contract**: verify required columns, validate types, and build fast “tripwires” that fail early.

Also: this dataset contains **PII** (names, mobile numbers, locations). We will:
- Create masked variants for any notebook display.
- Keep raw PII only in memory (no writing back to disk unless explicitly required).


In [ ]:
REQUIRED_COLS = [
    "Grievance Id",
    "Created Date",
    "Reported by User Name",
    "Mobile No.",
    "Complaint Location",
    "Complaint Subject",
    "Complaint Description",
    "Current Status",
    "Current Department Name",
    "Ward Name",
    "Current User Name",
    "Closing Remark",
]

missing_cols = [c for c in REQUIRED_COLS if c not in df.columns]
assert not missing_cols, f"Missing required columns: {missing_cols}"

# Quick type checks
assert df["Grievance Id"].notna().all(), "Grievance Id has nulls"
assert df["_created_at"].notna().all(), "Created Date parsing failed for some rows"

df.head(3)


## 3) PII‑safe view helpers

We mask:
- Mobile numbers → keep last 3 digits
- User names → initials only

This makes sharing screenshots / outputs safer.


In [ ]:
import hashlib

def mask_mobile(x):
    if pd.isna(x): return np.nan
    s = re.sub(r"\D+", "", str(x))
    if len(s) < 4: return "****"
    return "*"*(len(s)-3) + s[-3:]

def mask_name(x):
    if pd.isna(x): return np.nan
    parts = [p for p in re.split(r"\s+", str(x).strip()) if p]
    if not parts: return np.nan
    # initials: "Prashant Verat" -> "P. V."
    return " ".join([p[0].upper() + "." for p in parts[:3]])

def stable_hash(x, salt="nmmc"):
    if pd.isna(x): return np.nan
    s = f"{salt}::{str(x)}".encode("utf-8")
    return hashlib.sha256(s).hexdigest()[:12]

df_safe = df.copy()
df_safe["Mobile No."] = df_safe["Mobile No."].apply(mask_mobile)
df_safe["Reported by User Name"] = df_safe["Reported by User Name"].apply(mask_name)
df_safe["Current User Name"] = df_safe["Current User Name"].apply(mask_name)

df_safe.head(5)


## 4) Data quality report

Senior DS checklist:
- Missingness & “structural nulls”
- Uniqueness of primary keys
- Value distributions (categoricals)
- Timestamp integrity (range, timezone assumptions)
- Text fields: empty strings, extreme lengths


In [ ]:
# Primary key uniqueness
pk_dupes = df["Grievance Id"].duplicated().sum()
print("Duplicate Grievance Id rows:", pk_dupes)

# Missingness (%)
missing = (df.isna().mean()*100).sort_values(ascending=False)
missing.head(12)


In [ ]:
# Basic categorical distributions
for col in ["Current Status","Current Department Name","Ward Name"]:
    display(df[col].value_counts(dropna=False).head(15))


In [ ]:
# Text length sanity
text_cols = ["Complaint Subject","Complaint Description","Closing Remark","Complaint Location"]
for c in text_cols:
    s = df[c].astype(str).fillna("")
    lens = s.str.len()
    print(c, "min/median/p95/max:", int(lens.min()), int(lens.median()), int(lens.quantile(0.95)), int(lens.max()))


## 5) KPI Layer (operational analytics)

Even before AI, you can deliver value with a stable KPI layer:
- Ticket volume over time (daily/weekly)
- Closed vs open backlog
- Escalation & reopen rates
- Pareto charts by department/ward
- “Backlog age” (time since created) for non‑closed items

> Note: the file does **not** contain a resolved/closed timestamp. So resolution‑time SLAs cannot be computed directly.
If NMMC can export `Closed Date` / `Last Updated Date`, we can add SLA metrics and survival analysis.


In [ ]:
# Time buckets
df_kpi = df.copy()
df_kpi["date"] = df_kpi["_created_at"].dt.date
df_kpi["week"] = df_kpi["_created_at"].dt.to_period("W").astype(str)
df_kpi["month"] = df_kpi["_created_at"].dt.to_period("M").astype(str)

# Daily volume
daily = df_kpi.groupby("date")["Grievance Id"].size()

plt.figure()
daily.plot()
plt.title("Daily grievance volume")
plt.xlabel("Date")
plt.ylabel("Count")
plt.show()

daily.describe()


In [ ]:
# Status mix
status = df_kpi["Current Status"].value_counts()
plt.figure()
status.plot(kind="bar")
plt.title("Current status distribution")
plt.xlabel("Status")
plt.ylabel("Count")
plt.show()

(status / status.sum()).round(4)


In [ ]:
# Backlog (non-closed) and age buckets
asof = df_kpi["_created_at"].max()  # dataset as-of
open_mask = df_kpi["Current Status"].astype(str).str.upper().ne("CLOSED")
df_open = df_kpi.loc[open_mask].copy()
df_open["age_days"] = (asof - df_open["_created_at"]).dt.total_seconds() / 86400.0

df_open["age_bucket"] = pd.cut(
    df_open["age_days"],
    bins=[-1, 1, 3, 7, 14, 30, 60, 120, 99999],
    labels=["<=1d","1-3d","3-7d","7-14d","14-30d","30-60d","60-120d",">120d"]
)

display(df_open["age_bucket"].value_counts(dropna=False).sort_index())

plt.figure()
df_open["age_bucket"].value_counts().sort_index().plot(kind="bar")
plt.title("Open backlog age buckets (as-of max Created Date)")
plt.xlabel("Age bucket")
plt.ylabel("Count")
plt.show()


In [ ]:
# Pareto: top departments / wards
top_dept = df_kpi["Current Department Name"].value_counts().head(15)
top_ward = df_kpi["Ward Name"].value_counts()

plt.figure()
top_dept.plot(kind="bar")
plt.title("Top 15 departments by volume")
plt.xlabel("Department")
plt.ylabel("Count")
plt.show()

plt.figure()
top_ward.plot(kind="bar")
plt.title("Wards by volume")
plt.xlabel("Ward")
plt.ylabel("Count")
plt.show()


## 6) Text analytics (fast, explainable)

We start with an explainable baseline:
- Clean text (lowercase, strip punctuation, normalize whitespace)
- TF‑IDF features
- Topic discovery via **NMF** (interpretable topics)
- Clustering via **KMeans** to propose “sub‑topics”

These clusters are **not** final categories; they are an *assist* to:
1) propose a taxonomy  
2) speed up labeling  
3) become training data for a supervised classifier (or LLM prompt + eval set)


In [ ]:
import string

def clean_text(s):
    s = "" if pd.isna(s) else str(s)
    s = s.lower()
    s = s.replace("\n", " ")
    s = re.sub(r"\s+", " ", s).strip()
    # keep letters/numbers and basic punctuation; remove very noisy chars
    s = s.translate(str.maketrans("", "", string.punctuation))
    return s

df_text = df.copy()
df_text["text"] = (df_text["Complaint Subject"].fillna("").astype(str) + " " +
                   df_text["Complaint Description"].fillna("").astype(str)).map(clean_text)

df_text["text"].str.len().describe()


In [ ]:
# TF-IDF
vectorizer = TfidfVectorizer(
    max_features=40000,
    ngram_range=(1,2),
    min_df=5,
    max_df=0.8,
    stop_words="english"
)
X = vectorizer.fit_transform(df_text["text"])

X.shape


In [ ]:
# Topic modeling: NMF
n_topics = 12
nmf = NMF(n_components=n_topics, random_state=42, init="nndsvda", max_iter=400)
W = nmf.fit_transform(X)
H = nmf.components_

feature_names = np.array(vectorizer.get_feature_names_out())

def top_terms_for_topic(topic_idx, n=12):
    top_idx = np.argsort(H[topic_idx])[::-1][:n]
    return feature_names[top_idx].tolist()

topics = {f"topic_{i}": top_terms_for_topic(i, 14) for i in range(n_topics)}
pd.DataFrame({"topic": list(topics.keys()), "top_terms": list(topics.values())})


In [ ]:
# Cluster discovery (sub-topic candidates)
k = 18
km = KMeans(n_clusters=k, random_state=42, n_init=10)
clusters = km.fit_predict(W)

df_text["cluster"] = clusters

# For each cluster, show representative terms by averaging TF-IDF vectors
import scipy.sparse as sp

def cluster_top_terms(cluster_id, topn=15):
    idx = np.where(clusters == cluster_id)[0]
    if len(idx) == 0:
        return []
    sub = X[idx]
    mean_tfidf = np.asarray(sub.mean(axis=0)).ravel()
    top_idx = mean_tfidf.argsort()[::-1][:topn]
    return feature_names[top_idx].tolist()

cluster_summary = []
for c in range(k):
    n = int((df_text["cluster"]==c).sum())
    cluster_summary.append({
        "cluster": c,
        "n": n,
        "top_terms": cluster_top_terms(c, 16)
    })

cluster_df = pd.DataFrame(cluster_summary).sort_values("n", ascending=False)
cluster_df.head(12)


In [ ]:
# Show a few examples from the largest clusters (PII-safe display)
sample = (
    df_text.merge(df_safe[["Grievance Id","Reported by User Name","Mobile No."]], on="Grievance Id", how="left")
    .loc[:, ["Grievance Id","_created_at","Current Department Name","Ward Name","Current Status",
             "cluster","Reported by User Name","Mobile No.","Complaint Subject","Complaint Description"]]
)

for c in cluster_df.head(5)["cluster"]:
    display(sample[sample["cluster"]==c].head(5))


## 7) From clusters → real “Sub‑Topics” (human + LLM loop)

A robust taxonomy workflow (practical, senior DS approach):

1. **Seed taxonomy** from cluster summaries (above).
2. **Human review** (domain team) to merge/split/rename into business‑friendly sub‑topics.
3. **Create a gold eval set** (300–800 labeled rows): stratified by department/ward/status and cluster.
4. Use **Gemini** (or any LLM) to label sub‑topics with a strict schema:
   - `sub_topic`
   - `confidence`
   - `rationale` (short)
   - `needs_human_review` (bool)

5. Measure quality on eval set (accuracy/F1 + confusion analysis).  
6. Productionize with:
   - rules for low‑confidence → human queue
   - monitoring drift: top n-grams, embedding drift, volume drift, per-dept error rates

Below is a *prompt template* you can use for Gemini to propose sub‑topics consistently.


In [ ]:
GEMINI_SUBTOPIC_PROMPT = '''
You are an expert municipal grievance analyst.

TASK:
Given a grievance record (subject + description + department + ward), assign ONE sub-topic from the allowed list.

OUTPUT JSON ONLY (no markdown):
{
  "sub_topic": "<one of ALLOWED_SUB_TOPICS>",
  "confidence": <float 0..1>,
  "needs_human_review": <true|false>,
  "rationale": "<<=25 words>"
}

RULES:
- Choose exactly ONE sub_topic.
- If information is missing/ambiguous OR confidence < 0.65 => needs_human_review = true.
- Do NOT output personal data (names, phone numbers).
- Prefer business-meaningful buckets (actionable) over vague themes.

ALLOWED_SUB_TOPICS:
<PASTE_YOUR_FINAL_TAXONOMY_HERE>

RECORD:
- Department: {department}
- Ward: {ward}
- Subject: {subject}
- Description: {description}
'''.strip()

print(GEMINI_SUBTOPIC_PROMPT[:700] + "\n...")

## 8) Baseline supervised model (department prediction)

Why do this even if you use an LLM?
- Cheap, fast, stable baseline
- Detect outliers + routing errors
- Guardrail for LLM (disagreement checks)

We train a simple TF‑IDF + Logistic Regression classifier.


In [ ]:
# Prepare a compact dataset with enough text
ml = df_text.copy()
ml["y_dept"] = df["Current Department Name"].astype(str)

X = vectorizer.fit_transform(ml["text"])
y = ml["y_dept"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf = LogisticRegression(max_iter=2000, n_jobs=-1)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
print(classification_report(y_test, pred, zero_division=0))


## 9) Productization notes (what “senior DS” looks like)

### Minimum viable analytics layer (2–4 weeks)
- KPI Cockpit: volume, backlog, reopen/escalation, department/ward Pareto
- Cluster-driven sub-topic exploration
- Exportable “review queue” for humans (low-confidence / novel clusters)

### Minimum viable AI layer (4–8 weeks)
- Final sub-topic taxonomy (v1) + eval set
- Gemini labeler + human review loop + analytics on label distribution
- Monitoring: drift + per-ward/per-dept performance

### Data improvements to request from Probity/NMMC
To unlock SLA + accountability analytics:
- `Last Updated Date`
- `Closed Date` (or status history table)
- `Channel` (app/web/call center/WhatsApp/etc.)
- `Category` (if exists in source system)
- `Geo` (lat/long or standardized location codes)
- `Assignment history` (who/when it moved between users/depts)

### Governance / Privacy
- PII minimization in all downstream datasets
- Role-based access controls
- Hash IDs for join keys where possible
- Audit logs for data exports


## 10) Next steps checklist

1. Confirm target output: **Sub‑Topic taxonomy** vs **Department routing** vs both.  
2. Add `Closed Date` / status history export (critical for SLA).  
3. Create a 500‑row eval set (stratified by cluster + department + ward).  
4. Iterate taxonomy v1 → v2 using confusion analysis.  
5. Decide production path:
   - LLM-only (Gemini) with eval + monitoring
   - Hybrid: ML baseline + LLM fallback
   - Fully supervised after enough labels
